In [ ]:
import os

!/usr/share/locales/install-language-pack pt_BR.UTF-8
!dpkg-reconfigure locales
!pip install mlflow --quiet
!pip install utils --quiet


# Restart Python process to pick up the new locales
os.kill(os.getpid(), 9)

Generating locales (this might take a while)...
  pt_BR.UTF-8... done
Generation complete.
dpkg-trigger: error: must be called from a maintainer script (or with a --by-package option)

Type dpkg-trigger --help for help about this utility.
Generating locales (this might take a while)...
  en_US.UTF-8... done
  pt_BR.UTF-8... done
Generation complete.
     |████████████████████████████████| 16.9 MB 7.5 MB/s 
     |████████████████████████████████| 79 kB 8.7 MB/s 
     |████████████████████████████████| 77 kB 7.2 MB/s 
     |████████████████████████████████| 209 kB 54.8 MB/s 
     |████████████████████████████████| 181 kB 47.8 MB/s 
     |████████████████████████████████| 147 kB 61.9 MB/s 
     |████████████████████████████████| 78 kB 7.7 MB/s 
     |████████████████████████████████| 55 kB 3.1 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
     |████████████████████████████████| 140 kB 47.3 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from locale import atof, setlocale, LC_NUMERIC, LC_MONETARY
setlocale(LC_MONETARY, 'pt_BR.UTF-8')
import mlflow
from pprint import pprint
from mlflow import MlflowClient

In [2]:
# enable autologging
mlflow.sklearn.autolog()

In [3]:
def fetch_logged_data(run_id):
  """Fetch params, metrics, tags, and artifacts in the specified run"""
  client = MlflowClient()
  data = client.get_run(run_id).data
  # Exclude system tags: https://www.mlflow.org/docs/latest/tracking.html#system-tags
  tags = {k: v for k, v in data.tags.items() if not k.startswith("mlflow.")}
  artifacts = list(yield_artifacts(run_id))
  return {
      "params": data.params,
      "metrics": data.metrics,
      "tags": tags,
      "artifacts": artifacts,
  }

def yield_artifacts(run_id, path=None):
    """Yield all artifacts in the specified run"""
    client = MlflowClient()
    for item in client.list_artifacts(run_id, path):
        if item.is_dir:
            yield from yield_artifacts(run_id, item.path)
        else:
            yield item.path

In [4]:
df = pd.read_csv('/content/drive/MyDrive/ufpe/Eletivas/if697 ds/proj 2/serie_historica_precos_combustiveis-2021-02.csv', encoding = 'ISO-8859-1', sep=';', decimal=',')

In [5]:
columns = ['regiao', 'estado', 'municipio', 'revenda', 'cnpj', 'rua', 'numero', 'complemento', 'bairro', 'cep', 'produto', 'dtcoleta', 'vlvenda', 'vlcompra', 'medida', 'bandeira']
df.set_axis(columns, axis=1, inplace=True)

In [6]:
df = df.drop(columns=['estado', 'regiao', 'bandeira', 'medida', 'dtcoleta', 'bairro', 'cep', 'municipio', 'bairro', 'revenda', 'cnpj', 'rua', 'numero', 'complemento', 'vlcompra'])

In [7]:
from sklearn.model_selection import train_test_split

#dividir em 80(treino):10(teste):10(validação)
train_size=0.8

#A coluna selecionada para predição será produto
x = df.drop(columns = ['produto']).copy()
y = df['produto']

#obter dados de treinamento
x_train, x_rem, y_train, y_rem = train_test_split(x,y, train_size=0.8)

#agora dividir teste na metade do resto
test_size = 0.5
x_valid, x_test, y_valid, y_test = train_test_split(x_rem,y_rem, test_size=0.5)


In [8]:
def print_ml_stats():
  run_id = mlflow.last_active_run().info.run_id
  print("Logged data and model in run {}".format(run_id))

  # show logged data
  for key, data in fetch_logged_data(run_id).items():
      print("\n---------- logged {} ----------".format(key))
      pprint(data)

In [9]:
#Regressão Logística
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr = LogisticRegression(solver='saga', max_iter=200)
y_pred = lr.fit(x_train, y_train).predict(x_test)
print("Accuracy in test: " + str(accuracy_score(y_test, y_pred)))

2022/10/03 11:18:03 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'd2bbc23e3d9d42e19a4c0c61e9fbb4af', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy in test: 0.41354735016706845


In [10]:
print_ml_stats()

Logged data and model in run d2bbc23e3d9d42e19a4c0c61e9fbb4af

---------- logged params ----------
{'C': '1.0',
 'class_weight': 'None',
 'dual': 'False',
 'fit_intercept': 'True',
 'intercept_scaling': '1',
 'l1_ratio': 'None',
 'max_iter': '200',
 'multi_class': 'auto',
 'n_jobs': 'None',
 'penalty': 'l2',
 'random_state': 'None',
 'solver': 'saga',
 'tol': '0.0001',
 'verbose': '0',
 'warm_start': 'False'}

---------- logged metrics ----------
{'accuracy_score_x_test': 0.41354735016706845,
 'training_accuracy_score': 0.413361389855241,
 'training_f1_score': 0.315377860759953,
 'training_log_loss': 1.1730494000544482,
 'training_precision_score': 0.3035230250274077,
 'training_recall_score': 0.413361389855241,
 'training_roc_auc_score': 0.8020157833856082,
 'training_score': 0.413361389855241}

---------- logged tags ----------
{'estimator_class': 'sklearn.linear_model._logistic.LogisticRegression',
 'estimator_name': 'LogisticRegression'}

---------- logged artifacts ----------
['me

In [11]:
 #Gaussian NaiveBayes
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(x_train, y_train).predict(x_test)
print("Accuracy in test: " + str(accuracy_score(y_test, y_pred)))

2022/10/03 11:18:43 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'f4c1a8c3c2d44df890452c67b2629bbb', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy in test: 0.44072241255339845


In [12]:
print_ml_stats()

Logged data and model in run f4c1a8c3c2d44df890452c67b2629bbb

---------- logged params ----------
{'priors': 'None', 'var_smoothing': '1e-09'}

---------- logged metrics ----------
{'accuracy_score_x_test': 0.44072241255339845,
 'training_accuracy_score': 0.43786678791595735,
 'training_f1_score': 0.3851964726116846,
 'training_log_loss': 1.1531411300088636,
 'training_precision_score': 0.3855479813648383,
 'training_recall_score': 0.43786678791595735,
 'training_roc_auc_score': 0.8002345144019587,
 'training_score': 0.43786678791595735}

---------- logged tags ----------
{'estimator_class': 'sklearn.naive_bayes.GaussianNB',
 'estimator_name': 'GaussianNB'}

---------- logged artifacts ----------
['metric_info.json',
 'model/MLmodel',
 'model/conda.yaml',
 'model/model.pkl',
 'model/python_env.yaml',
 'model/requirements.txt',
 'training_confusion_matrix.png']


In [13]:
#Decision Tree
from sklearn import tree
clf = tree.DecisionTreeClassifier()
y_pred = clf.fit(x_train, y_train).predict(x_test)
print("Accuracy in test: " + str(accuracy_score(y_test, y_pred)))

2022/10/03 11:19:11 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'c8c72092eb8a4456a5fbfff79d3ad5f5', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


Accuracy in test: 0.4783656896332953


In [14]:
print_ml_stats()

Logged data and model in run c8c72092eb8a4456a5fbfff79d3ad5f5

---------- logged params ----------
{'ccp_alpha': '0.0',
 'class_weight': 'None',
 'criterion': 'gini',
 'max_depth': 'None',
 'max_features': 'None',
 'max_leaf_nodes': 'None',
 'min_impurity_decrease': '0.0',
 'min_samples_leaf': '1',
 'min_samples_split': '2',
 'min_weight_fraction_leaf': '0.0',
 'random_state': 'None',
 'splitter': 'best'}

---------- logged metrics ----------
{'accuracy_score_x_test': 0.4783656896332953,
 'training_accuracy_score': 0.4849187383024394,
 'training_f1_score': 0.4661746167166282,
 'training_log_loss': 1.069998039265622,
 'training_precision_score': 0.4894812123378349,
 'training_recall_score': 0.4849187383024394,
 'training_roc_auc_score': 0.8398987543051145,
 'training_score': 0.4849187383024394}

---------- logged tags ----------
{'estimator_class': 'sklearn.tree._classes.DecisionTreeClassifier',
 'estimator_name': 'DecisionTreeClassifier'}

---------- logged artifacts ----------
['metri

In [15]:
#SVM
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=3)
y_pred = clf.fit(x_train, y_train).predict(x_test)
print("Accuracy in test: " + str(accuracy_score(y_test, y_pred)))

2022/10/03 11:19:39 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '2b4146605417472eb45d84efa83660d2', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


Accuracy in test: 0.40834496468299286


In [16]:
print_ml_stats()

Logged data and model in run 2b4146605417472eb45d84efa83660d2

---------- logged params ----------
{'algorithm': 'auto',
 'leaf_size': '30',
 'metric': 'minkowski',
 'metric_params': 'None',
 'n_jobs': 'None',
 'n_neighbors': '3',
 'p': '2',
 'weights': 'uniform'}

---------- logged metrics ----------
{'accuracy_score_x_test': 0.40834496468299286,
 'training_accuracy_score': 0.4151378329509046,
 'training_f1_score': 0.41515771363520565,
 'training_log_loss': 9.705357507420564,
 'training_precision_score': 0.421378414788017,
 'training_recall_score': 0.4151378329509046,
 'training_roc_auc_score': 0.7174630346817606,
 'training_score': 0.4151378329509046}

---------- logged tags ----------
{'estimator_class': 'sklearn.neighbors._classification.KNeighborsClassifier',
 'estimator_name': 'KNeighborsClassifier'}

---------- logged artifacts ----------
['metric_info.json',
 'model/MLmodel',
 'model/conda.yaml',
 'model/model.pkl',
 'model/python_env.yaml',
 'model/requirements.txt',
 'trainin

In [17]:
df

,produto,vlvenda
0,GASOLINA ADITIVADA,5.699
1,GASOLINA,5.499
2,DIESEL S10,4.699
3,ETANOL,5.200
4,GASOLINA,5.490
...,...,...
472851,ETANOL,5.859
472852,GASOLINA,6.580
472853,ETANOL,4.740
472854,DIESEL S10,5.448
